In [1]:
! pip install tensorflow-gpu==2.2 -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import os

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


In [2]:
train_root_path ='./data/train_preliminary/'
train_ad_path = os.path.join(train_root_path,'ad.csv')
train_click_path = os.path.join(train_root_path,'click_log.csv')
train_user_path = os.path.join(train_root_path,'user.csv')

In [3]:
test_root_path = './data/test/'
test_ad_path = os.path.join(test_root_path,'ad.csv')
test_click_path = os.path.join(test_root_path,'click_log.csv')

In [4]:
df_click_train = pd.read_csv(train_click_path,dtype= {'creative_id':str,'click_times':str,'time':str})
df_click_test = pd.read_csv(test_click_path,dtype= {'creative_id':str,'click_times':str,'time':str})

In [5]:
df_ad_train = pd.read_csv(train_ad_path,na_values='\\N', 
                          dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )
df_ad_test = pd.read_csv(test_ad_path,na_values='\\N',
                         dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )

In [6]:
df_click_ad_train = df_click_train.merge(df_ad_train,on=['creative_id'],how='inner')
df_click_ad_test = df_click_test.merge(df_ad_test,on=['creative_id'],how='inner')

In [7]:
df_click_all =  pd.concat([df_click_ad_train,df_click_ad_test], axis=0)


In [8]:
df_click_all = df_click_all.sort_values('time', ascending=True)

In [9]:
df_click_all.fillna('un',inplace=True)

In [10]:
window=10
# embed_size = 128
embed_size = 200

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
class epoch_logger(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [12]:
aggcol = ['creative_id','ad_id','product_id','product_category','advertiser_id','industry','click_times','time']
func = lambda x: ' '.join(x.tolist())
doc = df_click_all.groupby(['user_id']).agg(
    {'creative_id':func,'ad_id':func,'product_id':func,'product_category':func,'advertiser_id':func,'industry':func,'click_times':func,'time':func}
)


In [13]:
doc = doc.reset_index()

In [14]:
maxlen = 80

In [15]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

new_df ={}
new_df['user_id'] = doc['user_id']
word_index ={}
tokenizer = None
for col in tqdm(aggcol):
    tokenizer = Tokenizer(lower=False, char_level=False, split=' ')
    tokenizer.fit_on_texts(doc[col])
    train_x = tokenizer.texts_to_sequences(doc[col])
    train_x = pad_sequences(train_x, maxlen=maxlen, value=0,padding='post')
    new_df[col] = list(train_x)
    word_index[col] =tokenizer.word_index
df_final = pd.DataFrame(new_df)
df_final.to_parquet('./model/cache/final_padding.parquet_8input',index=False)

100%|██████████| 8/8 [19:59<00:00, 149.95s/it]


In [16]:
import pickle
f = open("./model/cache/word_index.pkl","wb")
pickle.dump(word_index,f)
f.close()

In [22]:
func = lambda x: list(x)
# group_df = df_click_all.groupby(['user_id']).agg({'ad_id':func,'product_id':func,'product_category':func,'advertiser_id':func,'industry':func,'click_times':func,'time':func})
group_df = df_click_all.groupby(['user_id']).agg({'creative_id':func})

In [23]:
#product_docs = df_click_all.groupby(['user_id'])['product_id'].apply(list).reset_index(name='product_list')
# agg_col = ['ad_id','product_id','product_category','advertiser_id','industry','click_times','time']
# agg_col = ['ad_id','product_id','product_category','advertiser_id','industry']
agg_col = ['creative_id']
for col in agg_col:
    print(f'Start {col} embeding:')
#     w2v_model = Word2Vec(group_df[col], sg=1,window=window,min_count=1,size=embed_size,iter=5,compute_loss=True,callbacks=[epoch_logger()])
#     w2v_model.wv.save(f"./model/word2vec_{col}_128.model")
    w2v_model = Word2Vec(group_df[col], sg=1,window=window,min_count=1,size=embed_size,iter=10,workers=20, compute_loss=True,callbacks=[epoch_logger()])
    w2v_model.wv.save(f"./model/word2vec_{col}_200.model")

Start creative_id embeding:
Epoch #0 start
Loss after epoch 0: 38568428.0
Epoch #1 start
Loss after epoch 1: 19379080.0
Epoch #2 start
Loss after epoch 2: 10448660.0
Epoch #3 start
Loss after epoch 3: 2495392.0
Epoch #4 start
Loss after epoch 4: 2431224.0
Epoch #5 start
Loss after epoch 5: 2368592.0
Epoch #6 start
Loss after epoch 6: 2228960.0
Epoch #7 start
Loss after epoch 7: 2179912.0
Epoch #8 start
Loss after epoch 8: 2111304.0
Epoch #9 start
Loss after epoch 9: 2049736.0


# click_time 分箱处理


In [17]:
df_click_all['click_times'] = df_click_all['click_times'].astype(int)

In [18]:
#[1, 2, 3, 6, 10,19]
def buckize(x):
    if x ==1:
        return 1
    if x ==2:
        return 2
    if x >=3 and x<6:
        return 3
    if x >=6 and x<10:
        return 4
    if x >=10 and x<19:
        return 5
    return 6
df_click_all['click_times_buck'] = df_click_all['click_times'].apply(buckize)

In [19]:
df_click_time = df_click_all.groupby(['user_id'])['click_times_buck'].apply(list).reset_index()

In [20]:
df_click_time

,user_id,click_times_buck
0,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]"
1,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,5,"[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...
1899995,3999996,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, ..."
1899996,3999997,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1899997,3999998,"[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1899998,3999999,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ..."


In [22]:
train_click = pad_sequences(df_click_time['click_times_buck'], maxlen=80, value=0,padding='post')

In [23]:
new_df ={}
new_df['user_id'] = df_click_time['user_id']
new_df['click_times_buck'] =  list(train_click)

In [24]:
df_final = pd.DataFrame(new_df)

In [25]:
df_final.to_parquet('./model/cache/click_times_buck.parquet',index=False)

# Deepwalk

In [ ]:
def deepwalk(log,f1,f2,flag,L):
    #Deepwalk算法，
    print("deepwalk:",f1,f2)
    #构建图
    dic={}
    for item in log[[f1,f2]].values:
        try:
            str(int(item[1]))
            str(int(item[0]))
        except:
            continue
        try:
            dic['item_'+str(int(item[1]))].add('user_'+str(int(item[0])))
        except:
            dic['item_'+str(int(item[1]))]=set(['user_'+str(int(item[0]))])
        try:
            dic['user_'+str(int(item[0]))].add('item_'+str(int(item[1])))
        except:
            dic['user_'+str(int(item[0]))]=set(['item_'+str(int(item[1]))])
    dic_cont={}
    for key in dic:
        dic[key]=list(dic[key])
        dic_cont[key]=len(dic[key])
    print("creating")     
    #构建路径
    path_length=10        
    sentences=[]
    length=[]
    for key in dic:
        sentence=[key]
        while len(sentence)!=path_length:
            key=dic[sentence[-1]][random.randint(0,dic_cont[sentence[-1]]-1)]
            if len(sentence)>=2 and key == sentence[-2]:
                break
            else:
                sentence.append(key)
        sentences.append(sentence)
        length.append(len(sentence))
        if len(sentences)%100000==0:
            print(len(sentences))
    print(np.mean(length))
    print(len(sentences))
    #训练Deepwalk模型
    print('training...')
    random.shuffle(sentences)
    model = Word2Vec(sentences, size=L, window=4,min_count=1,sg=1, workers=10,iter=20)
    print('outputing...')
    #输出
    values=set(log[f1].values)
    w2v=[]
    for v in values:
        try:
            a=[int(v)]
            a.extend(model['user_'+str(int(v))])
            w2v.append(a)
        except:
            pass
    out_df=pd.DataFrame(w2v)
    names=[f1]
    for i in range(L):
        names.append(f1+'_'+ f2+'_'+names[0]+'_deepwalk_embedding_'+str(L)+'_'+str(i))
    out_df.columns = names
    print(out_df.head())
    out_df.to_pickle('data/' +f1+'_'+ f2+'_'+f1 +'_'+flag +'_deepwalk_'+str(L)+'.pkl') 
    ########################
    values=set(log[f2].values)
    w2v=[]
    for v in values:
        try:
            a=[int(v)]
            a.extend(model['item_'+str(int(v))])
            w2v.append(a)
        except:
            pass
    out_df=pd.DataFrame(w2v)
    names=[f2]
    for i in range(L):
        names.append(f1+'_'+ f2+'_'+names[0]+'_deepwalk_embedding_'+str(L)+'_'+str(i))
    out_df.columns = names
    print(out_df.head())
    out_df.to_pickle('data/' +f1+'_'+ f2+'_'+f2 +'_'+flag +'_deepwalk_'+str(L)+'.pkl') 
    